# 실습 2: Softmax Classification 만들기

## 1. 필요한 모듈 선언하기

In [22]:
import torch 
from torch import nn, optim, cuda
from torch.utils import data
import torch.nn.functional as F
import time
import torchvision
import torchvision.transforms as transforms
import numpy as np

## 2. Device 선언 및 랜덤 시드 고정

### 2-1) device 선언

In [23]:
device = 'cuda' if cuda.is_available() else 'cpu' # GPU 사용가능하면 사용, 없으면 CPU

### 2-2) 랜덤 시드 고정

In [24]:
torch.manual_seed(777)
if device=='cuda':
  torch.cuda.manual_seed_all(777)
# np.random.seed(777)


## 3. 데이터셋 불러오기¶

### 3-1) 이미지 전처리

In [25]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

### 3-2) 데이터셋 불러오기

In [26]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## 4. 모델 선언

### 4-1) class로 Model 만들기 

In [28]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(3072, 1024) 
        self.l2 = nn.Linear(1024, 512)
        self.l3 = nn.Linear(512, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 10) 

    def forward(self, x):
        x = x.view(-1,3072)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)

4-2) Model 선언

In [29]:
model = Net()
model.to(device)

Net(
  (l1): Linear(in_features=3072, out_features=1024, bias=True)
  (l2): Linear(in_features=1024, out_features=512, bias=True)
  (l3): Linear(in_features=512, out_features=240, bias=True)
  (l4): Linear(in_features=240, out_features=120, bias=True)
  (l5): Linear(in_features=120, out_features=10, bias=True)
)

## 5. 학습 시작

### 5-1) loss 및 optimizer 선언

In [36]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5) 

### 5-2) train, test 함수 

In [41]:
 # 학습
def train(epoch):
    model.train()
    # 모델을 학습 모드로 변환 
    for batch_idx, (data, target) in enumerate(trainloader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        
        loss = criterion(output, target) # torch.Size([4, 10]) torch.Size([4])
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('\nTrain Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(epoch, batch_idx * len(data), len(trainloader.dataset),100. * batch_idx / len(trainloader), loss.item()))


def test():
    model.eval()
    # 모델을 평가 모드로 변환 
    test_loss = 0
    correct = 0
    for data, target in testloader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    
    test_loss /= len(testloader.dataset)
    print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(testloader.dataset)} 'f'({100. * correct / len(testloader.dataset):.0f}%)')




In [42]:
if __name__ == '__main__':
    since = time.time()
    for epoch in range(1, 10):
        epoch_start = time.time() # 타이머 : 초.마이크로초
        train(epoch)
        m, s = divmod(time.time() - epoch_start, 60) #경과시간을 분과 초로 나타내려고
        print(f'Training time: {m:.0f}m {s:.0f}s')
        test()
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Testing time: {m:.0f}m {s:.0f}s')

    m, s = divmod(time.time() - since, 60) # 에폭 다 돌았을 때 전체 걸린 시간
    print(f'Total Time: {m:.0f}m {s:.0f}s\nModel was trained on {device}!')


Train Epoch: 1 | Batch Status: 0/50000 (0%) | Loss: 0.708796

Train Epoch: 1 | Batch Status: 12800/50000 (26%) | Loss: 0.504734

Train Epoch: 1 | Batch Status: 25600/50000 (51%) | Loss: 0.626783

Train Epoch: 1 | Batch Status: 38400/50000 (77%) | Loss: 0.556099
Training time: 0m 8s
Test set: Average loss: 0.0223, Accuracy: 5522/10000 (55%)
Testing time: 0m 10s

Train Epoch: 2 | Batch Status: 0/50000 (0%) | Loss: 0.638900

Train Epoch: 2 | Batch Status: 12800/50000 (26%) | Loss: 0.544315

Train Epoch: 2 | Batch Status: 25600/50000 (51%) | Loss: 0.489214

Train Epoch: 2 | Batch Status: 38400/50000 (77%) | Loss: 0.565880
Training time: 0m 8s
Test set: Average loss: 0.0227, Accuracy: 5488/10000 (55%)
Testing time: 0m 10s

Train Epoch: 3 | Batch Status: 0/50000 (0%) | Loss: 0.549894

Train Epoch: 3 | Batch Status: 12800/50000 (26%) | Loss: 0.548553

Train Epoch: 3 | Batch Status: 25600/50000 (51%) | Loss: 0.510154

Train Epoch: 3 | Batch Status: 38400/50000 (77%) | Loss: 0.452390
Training 